## Building the dataset that will be input into the model

In [8]:
import findspark
import pandas as pd
findspark.init()

from pyspark.sql import SparkSession
from pyspark import SparkConf

# for shared metastore (shared across all users)
spark = SparkSession.builder.appName("Building dataset").config("hive.metastore.uris", "thrift://bialobog:9083", conf=SparkConf()).getOrCreate() \

# for local metastore (your private, invidivual database) add the following config to spark session
spark.sql("USE 2023_04_01")

Py4JError: An error occurred while calling o316.toString. Trace:
java.lang.IllegalArgumentException: object is not an instance of declaring class
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.ClientServerConnection.run(ClientServerConnection.java:106)
	at java.lang.Thread.run(Thread.java:750)



In [5]:
import pyspark.pandas as ps
from pyspark.sql.functions import lit,col
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import xgboost as xgb
#from boruta import BorutaPy
#from fredapi import Fred
from sklearn.linear_model import Lasso
from sklearn.model_selection import TimeSeriesSplit
import csv
from pyspark.sql import functions as F
from functools import reduce
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf
from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, CrossValidatorModel
from CreateDataset import get_features_all_stocks


def get_macro_features():
    # fred_key = 'bdfdde3b7a21b7d528011d17996b0b8e'
    # fred = Fred(api_key=fred_key)
    # cpi = fred.get_series(series_id='CPIAUCSL')
    # cpi_change = cpi.pct_change()
    # unemp = fred.get_series(series_id='UNRATE')
    # gdp = fred.get_series(series_id='GDP')
    # gdp_change = gdp.pct_change()
    # df = pd.DataFrame({'CPI_change': cpi_change,'Unemployment_Rate': unemp,'GDP_change': gdp_change})
    # df.to_csv('macro.csv')
    df = pd.read_csv('macro.csv')
    return df


def train_test(df):
    df = df.filter(
        reduce(lambda acc, column: acc & (F.size(col(column)) == 23), df.columns[1:-1], lit(True))
        )
    # print(filtered_df.count())

    # average_lengths = df.agg(*[(F.avg(F.size(col(column))).alias(f'avg_length_{column}')) for column in df.columns[1:-1]])
    
    # test = padded_df.select('ff_non_oper_exp').filter(col('fsym_id')=='RTTY5P-R').collect()[0]
    # print(test['ff_non_oper_exp'], len(test['ff_non_oper_exp']))

    #need to decide whether to only include stocks that started from 2000, or include just from e.g. 2019
    #temporary measure - replace with 0
    #try imputer?
    #look into masking
    features = df.columns[1:-1]
    list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())
    for f in features:
        df = df.withColumn(f, list_to_vector_udf(f))
    df.show(2)
    vector_assembler = VectorAssembler(inputCols=features, outputCol="features_vector")
    df_assembled = vector_assembler.transform(df)
    
    lr = LogisticRegression(featuresCol="features_vector", labelCol="label")

    # pipeline = Pipeline(stages=[vector_assembler, lr])
    
    paramGrid = ParamGridBuilder() \
        .addGrid(lr.regParam, [0.01, 0.1]) \
        .addGrid(lr.elasticNetParam, [0.0, 0.5]) \
        .addGrid(lr.maxIter, [10, 20]) \
        .build()

    evaluator = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction", metricName="areaUnderROC")

    crossval = CrossValidator(estimator=lr,
                              estimatorParamMaps=paramGrid,
                              evaluator=evaluator,
                              numFolds=5) 

    cvModel = crossval.fit(df_assembled)

    avg_metrics = cvModel.avgMetrics

    for i, acc in enumerate(avg_metrics):
        print(f"Fold {i + 1} - Validation Accuracy: {acc}")

    best_model = cvModel.bestModel

    predictions = best_model.transform(df_assembled)
    predictions.select('fsym_id', 'label', 'prediction').show(100)
    tp = predictions.filter((predictions.label == 1) & (predictions.prediction == 1)).count()

    # Calculate True Negatives (TN)
    tn = predictions.filter((predictions.label == 0) & (predictions.prediction == 0)).count()

    # Calculate False Positives (FP)
    fp = predictions.filter((predictions.label == 0) & (predictions.prediction == 1)).count()

    # Calculate False Negatives (FN)
    fn = predictions.filter((predictions.label == 1) & (predictions.prediction == 0)).count()

    # Print the results
    print(f"True Positives: {tp}")
    print(f"True Negatives: {tn}")
    print(f"False Positives: {fp}")
    print(f"False Negatives: {fn}")

    # You can also calculate precision, recall, and F1-score if needed
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1_score = 2 * (precision * recall) / (precision + recall)

    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1_score}")

#     # Filter and display predictions for label 1
#     predictions.filter(predictions['label'] == 1).select("fsym_id", "label", "prediction").show()
# from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import roc_auc_score

def test_pandas(df1):
    df1 = df1.toPandas()
    exclude_columns = ['fsym_id', 'label']
    df = df1[df1.loc[:, ~df1.columns.isin(exclude_columns)].apply(lambda row: all(len(cell) == 23 for cell in row), axis=1)]
    print(len(df))
    X = df.drop(exclude_columns, axis=1)
    y = df['label']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    lr = LogisticRegression()

    param_grid = {
        'penalty': ['l1', 'l2'],
        'C': [0.01, 0.1, 1.0, 10.0],
        'max_iter': [100, 200, 300]
    }

    grid_search = GridSearchCV(lr, param_grid, cv=5, scoring='roc_auc')

    grid_search.fit(X_train, y_train)
    
    best_model = grid_search.best_estimator_

    predictions = best_model.predict(X_test)

    auc = roc_auc_score(y_test, predictions)

    print(f"Area under the ROC curve (AUC): {auc}")
    print("Best model hyperparameters:")
    print(grid_search.best_params_)
    
    #extra_df= df[df.apply(lambda row: any(len(cell) != 23 for cell in row), axis=1)]
    
    
    
    
#create_dataset('FF_ADVANCED_DER_AF')
df = get_features_all_stocks()
train_test(df)
#test_pandas(df)

+--------+--------------------+----------------------+--------------------+----------------------+--------------------+-----------------------+--------------------------+--------------------+--------------------+--------------------------+--------------------+-------------------------+-----------------------+--------------------+-----+
| fsym_id|     ff_non_oper_exp|ff_net_inc_bef_xord_gr|      ff_oper_inc_gr|ff_ut_non_oper_inc_oth|         ff_earn_yld|ff_net_inc_dil_aft_xord|ff_net_inc_basic_beft_xord|        ff_assets_gr|          ff_fcf_yld|ff_net_inc_dil_bef_unusual|            ff_wkcap|ff_net_inc_basic_aft_xord|ff_oper_inc_aft_unusual|      ff_net_inc_dil|label|
+--------+--------------------+----------------------+--------------------+----------------------+--------------------+-----------------------+--------------------------+--------------------+--------------------+--------------------------+--------------------+-------------------------+-----------------------+--------------

Py4JJavaError: An error occurred while calling o372.evaluate.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 166 in stage 3841.0 failed 4 times, most recent failure: Lost task 166.3 in stage 3841.0 (TID 109849) (128.16.11.216 executor 11): java.io.IOException: No space left on device
	at java.io.FileOutputStream.writeBytes(Native Method)
	at java.io.FileOutputStream.write(FileOutputStream.java:326)
	at org.apache.spark.storage.TimeTrackingOutputStream.write(TimeTrackingOutputStream.java:59)
	at org.apache.spark.io.MutableCheckedOutputStream.write(MutableCheckedOutputStream.scala:43)
	at java.io.BufferedOutputStream.flushBuffer(BufferedOutputStream.java:82)
	at java.io.BufferedOutputStream.flush(BufferedOutputStream.java:140)
	at net.jpountz.lz4.LZ4BlockOutputStream.finish(LZ4BlockOutputStream.java:265)
	at net.jpountz.lz4.LZ4BlockOutputStream.close(LZ4BlockOutputStream.java:195)
	at java.io.FilterOutputStream.close(FilterOutputStream.java:159)
	at java.io.Fil

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

model = Sequential()
model.add(SimpleRNN(units=32, input_shape=(23, input_features)))
model.add(Dense(units=output_features))



In [ ]:
spark.sql("SELECT MIN(a.p_date) FROM fp_basic_prices a LEFT JOIN sym_ticker_region s ON s.fsym_id = a.fsym_id WHERE s.ticker_region = 'AACQU-US' ").show()